# Exploratory Data Analysis (EDA)

### Import libraries

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

### Import data

In [2]:
df_res_ins = pd.read_csv('../dataset/restaurant_inspection_result_cleaned', index_col=0)
df_yelp = pd.read_csv('../dataset/yelp_nyc', index_col=0)

### Clean data

#### Inspection data

In [7]:
# Drop null values in building columns
df_res_ins.dropna(subset=['building'], inplace=True)

# Combine building and street columns to get the address
df_res_ins['address'] = df_res_ins.apply(lambda row: str(row.building) + str(" ") + str(row.street), axis=1)

# Change dba to name to match yelp data
df_res_ins.rename(columns={'dba': 'name'}, inplace=True)

# Make address all lower-case.
df_res_ins["address"] = [i.lower() for i in df_res_ins["address"]]

# Make phone all string
df_res_ins["phone"] = [str(i) for i in df_res_ins["phone"]]

#### yelp data

In [8]:
# Drop null values in address column
df_yelp.dropna(subset=['address'], inplace=True)

# Fill 0 in null value in phone column
df_yelp['phone'].fillna(0000000000, inplace=True)

# Make the name of the restaurant all lower-case.
df_yelp['name'] = [i.lower() for i in df_yelp["name"]]

# Make address all lower-case.
df_yelp['address'] = [str(i).lower() for i in df_yelp["address"]]

# Make phone all string
df_yelp['phone'] = [str(int(i))[1:11] for i in df_yelp["phone"]]

### Unify `address`  and `name` values in both dataset

In [9]:
# Apply the following translation to both of inspection data and yelp data.
# For address,

address_replace_dict = {' avenue': ' ave',
                        ' street': ' st',
                        ' west ': ' w ',
                        " east ": " e ",
                        "th ave": " ave",
                        "th st": " st",
                        "road": 'rd',
                        "lane": "ln",
                        "floor": 'fl',
                        'place': 'pl',
                        "1st": "1",
                        "2nd": "2",
                        "3rd": '3',
                        "-": "",
                        ".": ""}

for key in address_replace_dict:
    df_res_ins['address'] = [i.replace(key, address_replace_dict[key]) for i in df_res_ins['address']]
    df_yelp['address'] = [i.replace(key, address_replace_dict[key]) for i in df_yelp['address']]
    

In [10]:
# For restaurant name,

name_replace_dict = {"&": "and",
                    "new york": "ny",
                    " ice cream": "",
                    " chinese restarurant": "",
                    " sushi": ""} 

for key in name_replace_dict:
    df_res_ins['name'] = [i.replace(key, name_replace_dict[key]) for i in df_res_ins['name']]
    df_yelp['name'] = [i.replace(key, name_replace_dict[key]) for i in df_yelp['name']]
    

### Combine inspection data and yelp data by address, name and phone number

In [11]:
df_combined = df_res_ins.merge(df_yelp, how='inner', left_on=['address','name', 'phone', 'zipcode'], right_on=['address','name','phone', 'zip_code'])
df_combined

,camis,name,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,violation_code,violation_description,critical_flag,score,inspection_type,address,zip_code,rating,price,cuisine
0,30075445,morris park bake shop,bronx,1007,morris park ave,10462,7188924968,Bakery,2016-02-18,Violations were cited in the following area(s).,04L,Evidence of mice or live mice present in facil...,Critical,10.0,Cycle Inspection / Initial Inspection,1007 morris park ave,10462,4.0,$$,bakeries
1,30075445,morris park bake shop,bronx,1007,morris park ave,10462,7188924968,Bakery,2017-05-18,Violations were cited in the following area(s).,06D,"Food contact surface not properly washed, rins...",Critical,7.0,Cycle Inspection / Initial Inspection,1007 morris park ave,10462,4.0,$$,bakeries
2,30075445,morris park bake shop,bronx,1007,morris park ave,10462,7188924968,Bakery,2018-05-11,Violations were cited in the following area(s).,08C,Pesticide use not in accordance with label or ...,Not Critical,5.0,Cycle Inspection / Initial Inspection,1007 morris park ave,10462,4.0,$$,bakeries
3,40360076,carvel,brooklyn,203,church avenue,11218,7184389501,"Ice Cream, Gelato, Yogurt, Ices",2016-02-09,Violations were cited in the following area(s).,06C,Food not protected from potential source of co...,Critical,5.0,Cycle Inspection / Initial Inspection,203 church ave,11218,3.5,$,icecream
4,40360076,carvel,brooklyn,203,church avenue,11218,7184389501,"Ice Cream, Gelato, Yogurt, Ices",2017-03-16,Violations were cited in the following area(s).,10B,Plumbing not properly installed or maintained;...,Not Critical,6.0,Cycle Inspection / Initial Inspection,203 church ave,11218,3.5,$,icecream
5,40360076,carvel,brooklyn,203,church avenue,11218,7184389501,"Ice Cream, Gelato, Yogurt, Ices",2018-03-20,Violations were cited in the following area(s).,04K,Evidence of rats or live rats present in facil...,Critical,13.0,Cycle Inspection / Initial Inspection,203 church ave,11218,3.5,$,icecream
6,40361618,sal's deli,queens,12908,20 avenue,11356,7186619498,Delicatessen,2015-07-23,Violations were cited in the following area(s).,02G,Cold food item held above 41Âº F (smoked fish ...,Critical,21.0,Cycle Inspection / Initial Inspection,12908 20 ave,11356,4.5,$,delis
7,40361618,sal's deli,queens,12908,20 avenue,11356,7186619498,Delicatessen,2015-08-12,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,6.0,Cycle Inspection / Re-inspection,12908 20 ave,11356,4.5,$,delis
8,40361618,sal's deli,queens,12908,20 avenue,11356,7186619498,Delicatessen,2016-01-28,Violations were cited in the following area(s).,02B,Hot food item not held at or above 140Âº F.,Critical,18.0,Cycle Inspection / Initial Inspection,12908 20 ave,11356,4.5,$,delis
9,40361618,sal's deli,queens,12908,20 avenue,11356,7186619498,Delicatessen,2016-02-11,Violations were cited in the following area(s).,06C,Food not protected from potential source of co...,Critical,7.0,Cycle Inspection / Re-inspection,12908 20 ave,11356,4.5,$,delis


In [12]:
len(df_combined.camis.unique())

1388

### Save the data as csv

In [14]:
df_combined.to_csv('../dataset/final.csv')